In [1]:
path = '/notebook/kaggle/'
PROJ_HOME_DIR = path + 'data/ultrasound/'
DATA_HOME_DIR = PROJ_HOME_DIR # + 'sample/'

results_path = DATA_HOME_DIR + 'results/'
test_path = DATA_HOME_DIR + 'test/'
valid_path = DATA_HOME_DIR + 'valid/'
train_path = DATA_HOME_DIR + 'train/'

In [2]:
!pip install scikit-image
!pip install kaggle-cli

    100% |################################| 71kB 1.7MB/s ta 0:00:01
    100% |################################| 5.3MB 249kB/s eta 0:00:01
    100% |################################| 71kB 5.7MB/s eta 0:00:01
    100% |################################| 92kB 6.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for kaggle-cli ... done
  Stored in directory: /root/.cache/pip/wheels/16/28/c7/78658216416b7b481df91b485e43b6722daf59e71fc9cf9a58
  Running setup.py bdist_wheel for unicodecsv ... done
  Stored in directory: /root/.cache/pip/wheels/97/e2/16/219fa93b83edaff912b6805cfa19d0597e21f8d353f3e2d22f
  Running setup.py bdist_wheel for PrettyTable ... done
  Stored in directory: /root/.cache/pip/wheels/b6/90/7b/1c22b89217d0eba6d5f406e562365ebee804f0d4595b2bdbcd
  Running setup.py bdist_wheel for cmd2 ... done
  Stored in directory: /root/.cache/pip/wheels/aa/51/63/dfeef1471ea77d2e622021366ff563e07d5e95a502eb3eb19a
  Running setup.py bdist_wheel for pyperclip ... done
  Stored in directory: /roo

In [3]:
import json

with open(path+'kaggle.txt') as json_file:  
    kg_data = json.load(json_file)
    
username = kg_data['username']
password = kg_data['password']

In [4]:
!kg config -g -u $username -p $password -c 'ultrasound-nerve-segmentation'

'''
%cd $DATA_HOME_DIR
!kg download
'''

'\n%cd $DATA_HOME_DIR\n!kg download\n'

In [5]:
!echo $CUDA_VISIBLE_DEVICES

0


In [6]:
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

#import sys
#sys.path.append('/notebook/kaggle/data/ultrasound')
#from data import load_train_data, load_test_data

K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 96
img_cols = 96

Using TensorFlow backend.


In [7]:
def preprocess(imgs):
    ''' Convert images to np arrays of 8-bit unsigned integers and standardize shape '''
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

This competition is evaluated on the mean [Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient). The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth.

In [8]:
# Dice coefficient between actual and predicted pixels
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))

# Loss of Dice coefficient between actual and predicted pixels
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

Create and compile [**u-net model**](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/): a network that relies on the strong use of data augmentation to use the available annotated samples more efficiently. The architecture consists of a contracting path to capture context and a symmetric expanding path that enables precise localization.
[More info...](https://arxiv.org/abs/1505.04597)

In [9]:
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
model_checkpoint = ModelCheckpoint(results_path+'weights.h5', monitor='val_loss', save_best_only=True)

In [ ]:
# Load and preprocess train data
imgs_train = preprocess(np.load(DATA_HOME_DIR+'imgs_train.npy'))
imgs_mask_train = preprocess(np.load(DATA_HOME_DIR+'imgs_mask_train.npy'))

# Normalize data
imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # Mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255.  # scale masks to [0, 1]

/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Train model

In [ ]:
model.fit(imgs_train, imgs_mask_train, batch_size=32, epochs=20, verbose=1, shuffle=True,
          validation_split=0.2,
          callbacks=[model_checkpoint])

/usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:2289: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 4508 samples, validate on 1127 samples
Epoch 1/20
4508/4508 [==============================] - 20s - loss: -0.0247 - dice_coef: 0.0247 - val_loss: -0.0242 - val_dice_coef: 0.0242
Epoch 2/20
4508/4508 [==============================] - 17s - loss: -0.0981 - dice_coef: 0.0981 - val_loss: -0.2171 - val_dice_coef: 0.2171
Epoch 3/20
4508/4508 [==============================] - 17s - loss: -0.2261 - dice_coef: 0.2261 - val_loss: -0.2427 - val_dice_coef: 0.2427
Epoch 4/20
4508/4508 [==============================] - 17s - loss: -0.2533 - dice_coef: 0.2533 - val_loss: -0.2095 - val_dice_coef: 0.2095
Epoch 5/20
4508/4508 [==============================] - 18s - loss: -0.2699 - dice_coef: 0.2699 - val_loss: -0.2634 - val_dice_coef: 0.2634
Epoch 6/20
4508/4508 [==============================] - 18s - loss: -0.2942 - dice_coef: 0.2942 - val_loss: -0.3012 - val_dice_coef: 0.3012
Epoch 7/20
4508/4508 [==============================] - 17s - loss: -0.3178 - dice_coef: 0.3178 - val_loss: -0.2

Make predictions on test data

In [ ]:
# Load and preprocess test data
imgs_test = preprocess(np.load(DATA_HOME_DIR+'imgs_test.npy'))
imgs_id_test = np.load(DATA_HOME_DIR+'imgs_id_test.npy')

imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

In [ ]:
# Load saved weights
model.load_weights(results_path+'weights.h5')

# Predict masks using test data
imgs_mask_test = model.predict(imgs_test, verbose=1)
np.save('imgs_mask_test.npy', imgs_mask_test)

In [ ]:
# Save predicted masks to file
pred_dir = results_path+'preds'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for image, image_id in zip(imgs_mask_test, imgs_id_test):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

Generate submission file

In [ ]:
#from __future__ import print_function

#import numpy as np
from skimage.transform import resize

image_rows = 420
image_cols = 580

def prep(img):
    img = img.astype('float32')
    img = (img > 0.5).astype(np.uint8)  # threshold
    img = resize(img, (image_cols, image_rows), preserve_range=True)
    return img

def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])

# Create .csv for submission
argsort = np.argsort(imgs_id_test)
imgs_id_test = imgs_id_test[argsort]
imgs_test = imgs_test[argsort]

total = imgs_test.shape[0]
ids = []
rles = []
for i in range(total):
    img = imgs_test[i, 0]
    img = prep(img)
    rle = run_length_enc(img)

    rles.append(rle)
    ids.append(imgs_id_test[i])

    if i % 1000 == 0:
        print('{}/{}'.format(i, total))

first_row = 'img,pixels'
file_name = results_path+'submission.csv'

with open(file_name, 'w+') as f:
    f.write(first_row + '\n')
    for i in range(total):
        s = str(ids[i]) + ',' + rles[i]
        f.write(s + '\n')

In [ ]:
file_name = results_path+'submission.csv'
! kg submit $file_name